# VacationPy
----
### Now Boarding for The Mediterranean
The provided weather criteria (less than 10 mph winds, no clouds, and 70s temp sounded ideal, so I went with it!  Turns out, The Mediterranean area is the perfect spot.  Who doesn't love pasta and gyros?!

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data2 = pd.read_csv("output_data/cities.csv",index_col=[0])
weather_data2

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Padang,75,ID,1630195119,85,-0.9492,100.3543,73.45,3.09
1,San Francisco del Mar Viejo,83,MX,1630195214,70,16.2333,-94.6500,84.15,5.32
2,Hermanus,40,ZA,1630195215,69,-34.4187,19.2345,45.90,5.88
3,Katsuura,6,JP,1630195215,79,35.1333,140.3000,84.31,14.41
4,Atuona,27,PF,1630194933,75,-9.8000,-139.0333,78.12,23.17
...,...,...,...,...,...,...,...,...,...
575,Lakatoro,55,VU,1630195435,66,-16.0999,167.4164,80.19,12.30
576,Chipata,2,ZM,1630195435,32,-13.6333,32.6500,67.19,5.26
577,Killybegs,68,IE,1630195436,93,54.6333,-8.4500,59.45,5.75
578,Mehamn,100,NO,1630195436,78,71.0357,27.8492,44.92,10.13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_data2[["Lat", "Lng"]]
humidity = weather_data2["Humidity"]

# Plot Heatmap & heat layer
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=2.5)
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacay_data = weather_data2.loc[(weather_data2["Wind Speed"] <= 10) & (weather_data2["Cloudiness"] == 0) &
                                   (weather_data2["Max Temp"] >= 70) & (weather_data2["Max Temp"] <= 80)].dropna()
vacay_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
58,Palamós,0,ES,1630195254,49,41.8484,3.1291,76.10,4.00
97,Vao,0,NC,1630195271,77,-22.6667,167.4833,70.47,4.27
151,Alanya,0,TR,1630195218,53,36.5438,31.9998,79.43,2.26
284,Mitsamiouli,0,KM,1630195333,63,-11.3847,43.2844,75.54,7.11
295,Mount Isa,0,AU,1630195266,12,-20.7333,139.5000,74.97,0.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create a hotel_df
hotel_data = vacay_data.loc[:,["City","Country", "Lat", "Lng"]]

# Add Hotel column
hotel_data["Hotel Name"] = ""

hotel_data.head()

,City,Country,Lat,Lng,Hotel Name
58,Palamós,ES,41.8484,3.1291,
97,Vao,NC,-22.6667,167.4833,
151,Alanya,TR,36.5438,31.9998,
284,Mitsamiouli,KM,-11.3847,43.2844,
295,Mount Isa,AU,-20.7333,139.5000,


In [6]:
#API info
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [7]:
#loop for hotels closest to cities
for index, row in hotel_data.iterrows():
    # get city name, lat, lnt
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add params dict keyword with lat and lng
    params["location"] = f"{lat},{lng}"

    # make API call
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # put hotel info in Hotel Name column
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_data.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Result not found. Skipping...")
        
    print("-"*64)
    
print(f"End of results.")

Retrieving Results for Index 58: Palamós.
Closest hotel in Palamós is Hotel Cap Roig by Brava Hoteles.
----------------------------------------------------------------
Retrieving Results for Index 97: Vao.
Closest hotel in Vao is OURE TERA beach resort.
----------------------------------------------------------------
Retrieving Results for Index 151: Alanya.
Closest hotel in Alanya is PALMİYE BEACH HOTEL.
----------------------------------------------------------------
Retrieving Results for Index 284: Mitsamiouli.
Closest hotel in Mitsamiouli is Al Camar Lodge.
----------------------------------------------------------------
Retrieving Results for Index 295: Mount Isa.
Closest hotel in Mount Isa is ibis Styles Mt Isa Verona.
----------------------------------------------------------------
Retrieving Results for Index 314: Gold Coast.
Closest hotel in Gold Coast is Mantra Legends Surfers Paradise.
----------------------------------------------------------------
Retrieving Results for I

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
locations = hotel_data[["Lat", "Lng"]]

In [9]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))